In [1]:
%env TWELVE_LABS_API_KEY=tlk_1DFFA7P17P2G9G289EBM33Q40FR2

env: TWELVE_LABS_API_KEY=tlk_1DFFA7P17P2G9G289EBM33Q40FR2


In [8]:
%env GROQ_API_KEY=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

env: GROQ_API_KEY=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


In [75]:
%env GROQ_API_KEY=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

env: GROQ_API_KEY=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


In [2]:
from twelvelabs import TwelveLabs
import json
from io import StringIO
import requests
import pandas as pd
import numpy as np

import os
key = os.environ.get("TWELVE_LABS_API_KEY")
print(key)
client = TwelveLabs(api_key=key)


tlk_1DFFA7P17P2G9G289EBM33Q40FR2


In [3]:

def generate(vid_id:str)->str:
    BASE_URL = "https://api.twelvelabs.io/v1.2"
    data = {
        "video_id": "66ad42ce7b2deac81dd12825",
        "type": "summary",
        "prompt": "This video has a snowboarding competition. Generate a table that records one row per trick performed. Add one column to label which person performed each trick. If you do not know their names just refer to them as person-1 or person-2 as they appear sequentially in the video. In a second column, name the trick. In the third column note if the trick was a Success or a Failure. Keep your response brief and do not include anything aside from the table.",
        "temperature": 0.43
    }

    response = requests.post(f"{BASE_URL}/summarize", json=data, headers={"x-api-key": key})
    response_dict = json.loads(response.text)
    return response_dict['summary']

def get_temp_file_name(vid_id:str) -> str:
    """generate a temp name to store responses in"""
    return f"temp_{vid_id}.txt"

def store_gen(vid_id:str)->None:
    generate_text = generate(vid_id)
    with open(get_temp_file_name(vid_id),'w') as gen_holder:
        gen_holder.write(generate_text)

def get_local_gen(vid_id)->str:
    with open(get_temp_file_name(vid_id),'r') as file:
        return file.read()

def format_table(raw:str) -> pd.DataFrame:
    pipe_index = raw.find('|')
    out = raw[pipe_index+1:]
    table_io = StringIO(out)
    df = pd.read_csv(table_io,sep='|', skipinitialspace=True,engine='python')
    df.replace('-', np.nan, inplace=True)
    df.columns = df.columns.str.strip()
    return df

In [4]:
vid_id = "66ad42ce7b2deac81dd12825"
store_gen(vid_id)
gen_text = get_local_gen(vid_id)
df = format_table(gen_text)
df

,Person,Trick Description,Outcome,Unnamed: 3
NaN,----------,----------------------------------------------...,----------,NaN
NaN,Darcy,Back two in two out ...,Success,NaN
NaN,Liam,Front lip ...,Failure,NaN
NaN,Lane,Cap two pull ...,Failure,NaN
NaN,Darcy,Down flat down board slide ...,Success,NaN
NaN,Liam,Back tail same way 270 ...,Success,NaN
NaN,Lane,Front blunt same way grab mute on the rail ...,Success,NaN
NaN,Darcy,Switch front blunt same way ...,Success,NaN
NaN,Liam,Switch lip pretzel ...,Failure,NaN
NaN,Lane,Hard way cap two back to rag down rail ...,Success,NaN


In [90]:
groq_client = Groq(api_key=os.environ.get("GROQ_API_KEY") )
def trick_categories(transcript):
    completion = groq_client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {
                "role": "system",
                "content": "\nSystem Prompt\nvbnet\nCopy code\nYou are an AI trained to categorize snowboard tricks into predefined categories. The categories are [Grabs, Spins, Flips, Jumps, Slides, Buttering, Other]. For each trick or list of tricks provided, you will determine which categories each trick falls into and provide a binary indicator (1 or 0) for each category. If a trick falls into the 'Other' category, you will provide a new category name that you think captures the essence of the trick. The output for each trick should be in a json format with the keys being [Grabs, Spins, Flips, Jumps, Slides, Buttering, Other] and the values being 1 or 0 with the exception of Other which can hold a string. Do not provide a text explanation. Only return JSONs."
            },
            {
                "role": "user",
                "content": "Please categorize the following snowboard trick(s): 1. [\"Front board pretzel\",\"Back three, jump the whole rainbow, drag nose\"]\n"
            }
        ],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )
    
    for chunk in completion:
        print(chunk.choices[0].delta.content or "", end="")
        
trick_categories('test')

[
    {"Grabs": 1, "Spins": 0, "Flips": 0, "Jumps": 0, "Slides": 1, "Buttering": 0, "Other": 0},
    {"Grabs": 1, "Spins": 1, "Flips": 0, "Jumps": 1, "Slides": 0, "Buttering": 0, "Other": 0}
]

In [184]:
def trick_categories_table(transcript):
    
    from groq import Groq

    groq_client = Groq()
    
    transcript = list(transcript)
    system_prompt = """
You are an AI trained to categorize snowboard tricks into predefined categories. 
The categories are:

['Grabs', 'Spins', 'Flips', 'Jumps', 'Slides', 'Buttering', 'Other', 'New Category']

For each trick or list of tricks provided, you will determine which categories each trick falls into and provide a binary indicator (1 or 0) for each category. A trick
can fall into multiple categories. 
If the trick provided is invalid or null, then the binary indicator should be set to 0 for each category except for New Category, which should contain 'invalid input'. 
If a trick falls into the 'Other' category, you will provide a new category name that you think captures the essence of the trick in the New Category column.

The output columns should be: 

['Trick Name','Grabs', 'Spins', 'Flips', 'Jumps', 'Slides', 'Buttering', 'Other', 'New Category']
 
The output for each trick should be in a table format with a column for the trick and then a column for each category
and the values being 1 or 0 for all columns except for New Category, which is a string. Do not provide a text explanation or a text introduction. Only return a table. 
Keep going until all items in the list are processed. 


"""

    user_prompt = """
    Please categorize the following snowboard trick(s):

    [{transcript}]
    """.format(transcript=transcript)
    
    
    
    completion = groq_client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
        temperature=.5,
        max_tokens=1024,
        top_p=1,
        stream=False,
        stop=None,
        #response_format={"type": "json_object"}
        #stop=None,
    )
    
    #rint(completion.choices[0].message.content)
    
    #json_response =completion.choices[0].message.content
    #json_data = json.loads(json_response)
    
    
    # for chunk in completion:
    #     print(chunk.choices[0].delta.content or "", end="")

    return completion.choices[0].message.content

#df['Trick Description'].apply(trick_categories)

completion=trick_categories_table(df['Trick Description'])
completion

'Here is the categorized table for the provided snowboard tricks:\n\n| Trick Name | Grabs | Spins | Flips | Jumps | Slides | Buttering | Other | New Category |\n| --- | --- | --- | --- | --- | --- | --- | --- | --- |\n| Back two in two out | 0 | 0 | 0 | 0 | 1 | 0 | 0 |  |\n| Front lip | 0 | 0 | 0 | 0 | 1 | 0 | 0 |  |\n| Cap two pull | 0 | 0 | 0 | 0 | 1 | 0 | 0 |  |\n| Down flat down board slide | 0 | 0 | 0 | 0 | 1 | 0 | 0 |  |\n| Back tail same way 270 | 0 | 1 | 0 | 0 | 1 | 0 | 0 |  |\n| Front blunt same way grab mute on the rail | 1 | 0 | 0 | 0 | 1 | 0 | 0 |  |\n| Switch front blunt same way | 0 | 0 | 0 | 0 | 1 | 0 | 0 |  |\n| Switch lip pretzel | 0 | 0 | 0 | 0 | 1 | 0 | 0 |  |\n| Hard way cap two back to rag down rail | 0 | 0 | 0 | 0 | 1 | 0 | 0 |  |\n| Switch front blunt same way | 0 | 0 | 0 | 0 | 1 | 0 | 0 |  |\n| Front board pretzel | 0 | 0 | 0 | 0 | 1 | 0 | 0 |  |\n| Back three, jump the whole rainbow, drag nose | 0 | 1 | 0 | 1 | 0 | 0 | 0 |  |\n| Switch lip pretzel | 0 | 0 | 0 |

In [262]:
from io import StringIO

def format_table(raw:str) -> pd.DataFrame:
    pipe_index = raw.find('|')
    out = raw[pipe_index+1:]
    table_io = StringIO(out)
    df = pd.read_csv(table_io,sep='|', skipinitialspace=True,engine='python')
    df.replace('-', np.nan, inplace=True)
    df.columns = df.columns.str.strip()
    return df

trick_df = format_table(completion) 


In [264]:
trick_df = trick_df[['Trick Name','Grabs', 'Spins', 'Flips', 'Jumps', 'Slides', 'Buttering',
       'Other']]
trick_df[['Grabs', 'Spins', 'Flips', 'Jumps', 'Slides', 'Buttering',
       'Other']]= trick_df[['Grabs', 'Spins', 'Flips', 'Jumps', 'Slides', 'Buttering',
       'Other']].apply(lambda row: pd.to_numeric(row,errors='coerce'),axis=0)
trick_df

,Trick Name,Grabs,Spins,Flips,Jumps,Slides,Buttering,Other
NaN,---,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,Back two in two out,0.0,0.0,0.0,0.0,1.0,0.0,0.0
NaN,Front lip,0.0,0.0,0.0,0.0,1.0,0.0,0.0
NaN,Cap two pull,0.0,0.0,0.0,0.0,1.0,0.0,0.0
NaN,Down flat down board slide,0.0,0.0,0.0,0.0,1.0,0.0,0.0
NaN,Back tail same way 270,0.0,1.0,0.0,0.0,1.0,0.0,0.0
NaN,Front blunt same way grab mute on the rail,1.0,0.0,0.0,0.0,1.0,0.0,0.0
NaN,Switch front blunt same way,0.0,0.0,0.0,0.0,1.0,0.0,0.0
NaN,Switch lip pretzel,0.0,0.0,0.0,0.0,1.0,0.0,0.0
NaN,Hard way cap two back to rag down rail,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [310]:
subcategories_dict = {
    'Slides': {'description':
f"""Boardslide: Sliding with the board perpendicular to the rail.
Lipslide: Approaching from the opposite side and sliding with the board perpendicular to the rail.
50-50: Sliding straight along a rail with the board parallel.
Nose Slide: Sliding on the nose of the board.
Tail Slide: Sliding on the tail of the board.
Blunt Slide: Sliding on the tail of the board with the nose raised.""",

'columns': f"""[Trick Name, Boardslide, Lipslide, 50-50, Nose Slide, Tail Slide, Blunt Slide]"""},

    'Spins': {'description':
f"""180: A half-spin, rotating 180 degrees.
360: A full spin, rotating 360 degrees.
540: A spin with one and a half rotations.
720: A double spin with two full rotations.
900: Two and a half rotations.
1080: Three full rotations.
1260: Three and a half rotations.
1440: Four full rotations.
1620: 4.5 rotations.
1800: 5 rotations.
1980: 5.5 rotations.
2160: 6 rotations
""",

'columns': f"""[Trick Name, 180, 360, 540, 720, 900, 1080, 1260, 1440, 1620, 1800, 1980, 2160]"""},
 
#removed grabs 
#Indy: Grabbing the toe edge of the board between the bindings with the back hand.
#Mute: Grabbing the toe edge of the board between the bindings with the front hand.
    
'Grabs': {'description':
f"""
Stalefish: Grabbing the heel edge of the board between the bindings with the back hand.
Melon: Grabbing the heel edge of the board between the bindings with the front hand.
Tail Grab: Grabbing the tail of the board.
Nose Grab: Grabbing the nose of the board.
Method: Grabbing the heel edge of the board with the front hand while arching the back and extending the legs.
Japan: Grabbing the toe edge of the board with the front hand while tucking the knees and rotating the board.
Seatbelt: Grabbing the nose of the board with the back hand.
Truck Driver: Grabbing the nose of the board with the front hand and the tail with the back hand simultaneously.""",

'columns': f"""[Trick Name, Indy, Mute, Stalefish, Melon, Tail Grab, Nose Grab, Method, Japan, Seatbelt, Truck Driver]"""},

'Flips': {'description':
f"""Backflip: A backward somersault.
Frontflip: A forward somersault.
Double Backflip: Two backward somersaults.
Double Frontflip: Two forward somersaults.
Rodeo: A backward flip with a 180 or 540-degree spin.
Misty: A forward flip with a 180 or 540-degree spin.
Cork: An off-axis spin, where the boarder is tilted.
Double Cork: A double off-axis spin.
Wildcat: A backflip with a rotation around the snowboarder’s side.""",

'columns': f"""[Trick Name, Backflip, Frontflip, Double Backflip, Double Frontflip, Rodeo, Misty, Cork, Double Cork, Wildcat]"""},

'Jumps': {'description':
f"""Ollie: Lifting the front foot, followed by the back foot, to jump.
Nollie: Lifting the back foot, followed by the front foot, to jump.
""",

'columns': f"""[Trick Name, Ollie, Nollie]"""},

'Buttering': {'description':
f"""Nose Butter: Pressing down on the nose of the board while rotating the tail.
Tail Butter: Pressing down on the tail of the board while rotating the nose.
Nose Roll: A 180-degree rotation while buttering on the nose.
Tail Roll: A 180-degree rotation while buttering on the tail.

""",

'columns': f"""[Trick Name, Nose Butter, Nose Roll, Tail Roll]"""}

}

In [311]:
# category = 'Buttering'
# transcript = trick_df[trick_df[category]==1].drop_duplicates(subset='Trick Name')['Trick Name']
# transcript = list(transcript)
# subcategories_description = subcategories_dict[category]['description']

# print(subcategories_description)



# subcategories_columns = subcategories_dict[category]['columns']

# print(subcategories_columns)
# system_prompt = """
# You are an AI trained to categorize snowboard tricks into predefined categories. 
# The categories are:

# {subcategories_description}


# For each trick or list of tricks provided, you will determine which categories each trick falls into and provide a binary indicator (1 or 0) for each category. A trick
# can fall into multiple categories. 
# If the trick provided is invalid or null, then the binary indicator should be set to 0 for each category. 


# The output columns should be: 

# {subcategories_columns}
 
# The output for each trick should be in a table format with a column for the trick and then a column for each category
# and the values being 1 or 0 for all columns.
# Do not provide a text explanation or a text introduction. Only return a table. 
# Keep going until all items in the list are processed. 


# """.format(subcategories_description=subcategories_description, subcategories_columns=subcategories_columns)

# user_prompt = """
# Please categorize the following snowboard trick(s):

# [{transcript}]
# """.format(transcript=transcript)



# completion = groq_client.chat.completions.create(
#     model="llama3-70b-8192",
#     messages=[
#     {"role": "system", "content": system_prompt},
#     {"role": "user", "content": user_prompt},
# ],
#     temperature=.4,
#     max_tokens=1024,
#     top_p=1,
#     stream=False,
#     stop=None,
# )
    
# formatted_table = format_table(completion.choices[0].message.content)
# formatted_table
    


Nose Butter: Pressing down on the nose of the board while rotating the tail.
Tail Butter: Pressing down on the tail of the board while rotating the nose.
Nose Roll: A 180-degree rotation while buttering on the nose.
Tail Roll: A 180-degree rotation while buttering on the tail.


[Trick Name, Nose Butter, Nose Roll, Tail Roll]


,Trick Name,Nose Butter,Nose Roll,Tail Butter,Tail Roll,Unnamed: 5
NaN,---,---,---,---,---,NaN
NaN,NaN,0,0,0,0,NaN


In [312]:
def subcategories_table(trick_df,subcategories_dict,category):
    
    from groq import Groq

    groq_client = Groq()
    
    subcategories_description = subcategories_dict[category]['description']
    subcategories_columns = subcategories_dict[category]['columns']
    
    
    transcript = trick_df[trick_df[category]==1].drop_duplicates(subset='Trick Name')['Trick Name']
    transcript = list(transcript)
    system_prompt = """
You are an AI trained to categorize snowboard tricks into predefined categories. 
The categories are:

{subcategories_description}


For each trick or list of tricks provided, you will determine which categories each trick falls into and provide a binary indicator (1 or 0) for each category. A trick
can fall into multiple categories. 
If the trick provided is invalid or null, then the binary indicator should be set to 0 for each category. 


The output columns should be: 

{subcategories_columns}
 
The output for each trick should be in a table format with a column for the trick and then a column for each category
and the values being 1 or 0 for all columns. 
Do not provide a text explanation or a text introduction. Only return a table. 
Keep going until all items in the list are processed. 


""".format(subcategories_description=subcategories_description, subcategories_columns=subcategories_columns)

    user_prompt = """
    Please categorize the following snowboard trick(s):

    [{transcript}]
    """.format(transcript=transcript)
    
    
    
    completion = groq_client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
        temperature=.5,
        max_tokens=1024,
        top_p=1,
        stream=False,
        stop=None,
    )
    
    formatted_table = format_table(completion.choices[0].message.content)
    numcols = [col for col in formatted_table.columns if 'Trick Name' not in col]
    dropcols = [col for col in formatted_table.columns if 'Unnamed' in col]
    sub_cols = [col for col in numcols if 'Unnamed' not in col]
    
    if 'Trick Name' not in formatted_table.columns:
        #this is an LLM error
        formatted_table['Trick Name'] = transcript
        print('added trick names')
        
    trick_df=trick_df.merge(formatted_table,on='Trick Name', how='left').fillna(0)
    trick_df[numcols] = trick_df[numcols].apply(lambda row: pd.to_numeric(row, errors='coerce'),axis=1).fillna(0)
    
    trick_df = trick_df.drop(columns=dropcols,errors='ignore')
    

    return trick_df, sub_cols


main_cats = list(subcategories_dict.keys())
trick_df2 = trick_df.copy()


cat_dict = {}

for cat in main_cats:
    print(cat)
    trick_df2, sub_cols = subcategories_table(trick_df2,subcategories_dict=subcategories_dict,category=cat)
    cat_dict[cat] = sub_cols
    

trick_df2

Slides
Spins
Grabs
Flips
Jumps
Buttering


,Trick Name,Grabs,Spins,Flips,Jumps,Slides,Buttering,Other,Boardslide,Lipslide,...,Misty,Cork,Double Cork,Wildcat,Ollie,Nollie,Nose Butter,Nose Roll,Tail Butter,Tail Roll
0,---,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Back two in two out,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Front lip,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Cap two pull,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Down flat down board slide,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Back tail same way 270,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Front blunt same way grab mute on the rail,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Switch front blunt same way,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Switch lip pretzel,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Hard way cap two back to rag down rail,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [333]:
trick_df2 = trick_df2.fillna(0)
for cat in list(subcategories_dict.keys()):
    sub_cols = cat_dict[cat]
    trick_df2['Uncategorized_'+cat] = trick_df2.apply(
    lambda row: 1 if row[cat] == 1 and all(row[sub_cols] == 0) else 0,
    axis=1
)

#get rid of the main categories. Now we only have subcategories    
trick_df3 = trick_df2.drop(columns=list(subcategories_dict.keys()))
trick_df3


,Trick Name,Other,Boardslide,Lipslide,50-50,Nose Slide,Tail Slide,Blunt Slide,180,360,...,Nose Butter,Nose Roll,Tail Butter,Tail Roll,Uncategorized_Slides,Uncategorized_Spins,Uncategorized_Grabs,Uncategorized_Flips,Uncategorized_Jumps,Uncategorized_Buttering
0,---,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1,Back two in two out,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,0,0,0,0,0
2,Front lip,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
3,Cap two pull,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,0,0,0,0,0
4,Down flat down board slide,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
5,Back tail same way 270,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
6,Front blunt same way grab mute on the rail,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
7,Switch front blunt same way,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
8,Switch lip pretzel,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
9,Hard way cap two back to rag down rail,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,0,0,0,0,0



# Scoring

In [319]:
cols= [col for col in trick_df3.columns if 'Trick Name' not in col]
cols

['Other',
 'Boardslide',
 'Lipslide',
 '50-50',
 'Nose Slide',
 'Tail Slide',
 'Blunt Slide',
 '180',
 '360',
 '540',
 '720',
 '900',
 '1080',
 '1260',
 '1440',
 '1620',
 '1800',
 '1980',
 '2160',
 'Indy',
 'Mute',
 'Stalefish',
 'Melon',
 'Tail Grab',
 'Nose Grab',
 'Method',
 'Japan',
 'Seatbelt',
 'Truck Driver',
 'Backflip',
 'Frontflip',
 'Double Backflip',
 'Double Frontflip',
 'Rodeo',
 'Misty',
 'Cork',
 'Double Cork',
 'Wildcat',
 'Ollie',
 'Nollie',
 'Nose Butter',
 'Nose Roll',
 'Tail Butter',
 'Tail Roll',
 'Uncategorized_Slides',
 'Uncategorized_Spins',
 'Uncategorized_Grabs',
 'Uncategorized_Flips',
 'Uncategorized_Jumps',
 'Uncategorized_Buttering']

In [330]:
scoring_dict = {
    # Grabs
    'Indy': 100,
    'Mute': 100,
    'Stalefish': 150,
    'Melon': 150,
    'Tail Grab': 200,
    'Nose Grab': 200,
    'Method': 250,
    'Japan': 250,
    'Seatbelt': 300,
    'Truck Driver': 400,
    'Uncategorized_Grabs': 100,

    # Spins
    '180': 50,
    '360': 100,
    '540': 150,
    '720': 200,
    '900': 300,
    '1080': 400,
    '1260': 500,
    '1440': 600,
    '1620': 700,
    '1800': 800,
    '1980': 900,
    '2160': 1000,

    # Flips
    'Backflip': 200,
    'Frontflip': 200,
    'Double Backflip': 400,
    'Double Frontflip': 400,
    'Rodeo': 300,
    'Misty': 300,
    'Cork': 350,
    'Double Cork': 700,
    'Wildcat': 250,

    # Jumps
    'Ollie': 50,
    'Nollie': 50,

    # Slides and Grinds
    'Boardslide': 100,
    'Lipslide': 150,
    '50-50': 100,
    'Nose Slide': 150,
    'Tail Slide': 150,
    'Blunt Slide': 200,

    # Buttering
    'Nose Butter': 100,
    'Tail Butter': 100,
    'Nose Roll': 150,
    'Tail Roll': 150,

    # Uncategorized Tricks
    'Uncategorized_Grabs': 100,
    'Uncategorized_Spins': 50,
    'Uncategorized_Flips': 200,
    'Uncategorized_Jumps': 50,
    'Uncategorized_Slides': 100,
    'Uncategorized_Buttering': 100,
    
    'Other': 100
}


In [351]:
trick_df3['num_tricks'] = trick_df3[scoring_cols].sum(axis=1)
scoring_df = trick_df3.copy()
scoring_cols = list(scoring_df.columns.drop(['Trick Name','num_tricks']))



for trick in scoring_cols:
    score = scoring_dict[trick]
    scoring_df[trick] = scoring_df[trick] * score

multiplier = (np.maximum(scoring_df['num_tricks'],1) - 1)*0.5 + 1
scoring_df['multiplier'] = multiplier
scoring_df['score'] = scoring_df[scoring_cols].sum(axis=1)*multiplier
scoring_df

,Trick Name,Other,Boardslide,Lipslide,50-50,Nose Slide,Tail Slide,Blunt Slide,180,360,...,Tail Roll,Uncategorized_Slides,Uncategorized_Spins,Uncategorized_Grabs,Uncategorized_Flips,Uncategorized_Jumps,Uncategorized_Buttering,num_tricks,multiplier,score
0,---,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0.0,1.0,0.0
1,Back two in two out,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,100,0,0,0,0,0,1.0,1.0,100.0
2,Front lip,0.0,0.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,1.0,1.0,150.0
3,Cap two pull,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,100,0,0,0,0,0,1.0,1.0,100.0
4,Down flat down board slide,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,1.0,1.0,100.0
5,Back tail same way 270,0.0,0.0,0.0,0.0,0.0,150.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,2.0,1.5,450.0
6,Front blunt same way grab mute on the rail,0.0,0.0,0.0,0.0,0.0,0.0,200.0,0.0,0.0,...,0.0,0,0,0,0,0,0,2.0,1.5,450.0
7,Switch front blunt same way,0.0,0.0,0.0,0.0,0.0,0.0,200.0,0.0,0.0,...,0.0,0,0,0,0,0,0,1.0,1.0,200.0
8,Switch lip pretzel,0.0,0.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,1.0,1.0,150.0
9,Hard way cap two back to rag down rail,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,100,0,0,0,0,0,1.0,1.0,100.0


In [353]:
total_score = scoring_df['score'].sum()
total_score

np.float64(4975.0)

In [354]:
scoring_summary = scoring_df.sort_values(by='score',ascending=False)[['Trick Name','score']]
scoring_summary

,Trick Name,score
22,"Half cab, gnarly bag, rodeo five",800.0
5,Back tail same way 270,450.0
6,Front blunt same way grab mute on the rail,450.0
12,"Back three, jump the whole rainbow, drag nose",375.0
10,Switch front blunt same way,200.0
7,Switch front blunt same way,200.0
23,"Switch 50, slow back three",200.0
27,Switch back lip,150.0
13,Switch lip pretzel,150.0
8,Switch lip pretzel,150.0


In [365]:
scoring_df

,Trick Name,Other,Boardslide,Lipslide,50-50,Nose Slide,Tail Slide,Blunt Slide,180,360,...,Tail Roll,Uncategorized_Slides,Uncategorized_Spins,Uncategorized_Grabs,Uncategorized_Flips,Uncategorized_Jumps,Uncategorized_Buttering,num_tricks,multiplier,score
0,---,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0.0,1.0,0.0
1,Back two in two out,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,100,0,0,0,0,0,1.0,1.0,100.0
2,Front lip,0.0,0.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,1.0,1.0,150.0
3,Cap two pull,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,100,0,0,0,0,0,1.0,1.0,100.0
4,Down flat down board slide,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,1.0,1.0,100.0
5,Back tail same way 270,0.0,0.0,0.0,0.0,0.0,150.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,2.0,1.5,450.0
6,Front blunt same way grab mute on the rail,0.0,0.0,0.0,0.0,0.0,0.0,200.0,0.0,0.0,...,0.0,0,0,0,0,0,0,2.0,1.5,450.0
7,Switch front blunt same way,0.0,0.0,0.0,0.0,0.0,0.0,200.0,0.0,0.0,...,0.0,0,0,0,0,0,0,1.0,1.0,200.0
8,Switch lip pretzel,0.0,0.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,1.0,1.0,150.0
9,Hard way cap two back to rag down rail,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,100,0,0,0,0,0,1.0,1.0,100.0


In [371]:
best_trick = scoring_df.sort_values(by='score',ascending=False).iloc[0,1:].reset_index()
trick_name = scoring_df.sort_values(by='score',ascending=False).iloc[0,0]
print(trick_name)
best_trick.columns = ['Trick','Score']
best_trick[best_trick['Score'] > 0]



Half cab, gnarly bag, rodeo five 


,Trick,Score
7,180,50.0
33,Rodeo,300.0
48,Uncategorized_Jumps,50
50,num_tricks,3.0
51,multiplier,2.0
52,score,800.0


In [206]:
def slide_categories_table(transcript):
    
    from groq import Groq

    groq_client = Groq()
    
    transcript = list(transcript)
    system_prompt = """
You are an AI trained to categorize snowboard tricks into predefined categories. 
The categories are:

Boardslide: Sliding with the board perpendicular to the rail.
Lipslide: Approaching from the opposite side and sliding with the board perpendicular to the rail.
50-50: Sliding straight along a rail with the board parallel.
Nose Slide: Sliding on the nose of the board.
Tail Slide: Sliding on the tail of the board.
Blunt Slide: Sliding on the tail of the board with the nose raised.


For each trick or list of tricks provided, you will determine which categories each trick falls into and provide a binary indicator (1 or 0) for each category. A trick
can fall into multiple categories. 
If the trick provided is invalid or null, then the binary indicator should be set to 0 for each category. 


The output columns should be: 

[Trick Name, Boardslide, Lipslide, 50-50, Nose Slide, Tail Slide, Blunt Slide]
 
The output for each trick should be in a table format with a column for the trick and then a column for each category
and the values being 1 or 0 for all columns. Do not provide a text explanation or a text introduction. Only return a table. 
Keep going until all items in the list are processed. 


"""

    user_prompt = """
    Please categorize the following snowboard trick(s):

    [{transcript}]
    """.format(transcript=transcript)
    
    
    
    completion = groq_client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
        temperature=.5,
        max_tokens=1024,
        top_p=1,
        stream=False,
        stop=None,
    )

    return completion.choices[0].message.content



slide_cats=slide_categories_table(df['Trick Description'])
slide_cats = format_table(slide_cats) 


slide_cats

,Trick Name,Boardslide,Lipslide,50-50,Nose Slide,Tail Slide,Blunt Slide,Unnamed: 7
NaN,---,---,---,---,---,---,---,NaN
NaN,Back two in two out,0,0,0,0,0,0,NaN
NaN,Front lip,0,1,0,0,0,0,NaN
NaN,Cap two pull,0,0,0,0,0,0,NaN
NaN,Down flat down board slide,1,0,0,0,0,0,NaN
NaN,Back tail same way 270,0,0,0,0,1,0,NaN
NaN,Front blunt same way grab mute on the rail,0,0,0,0,0,1,NaN
NaN,Switch front blunt same way,0,0,0,0,0,1,NaN
NaN,Switch lip pretzel,0,1,0,0,0,0,NaN
NaN,Hard way cap two back to rag down rail,0,0,0,0,0,0,NaN


In [211]:
slide_numcols = [col for col in slide_cats.columns if 'Trick' not in col]
slide_cats[slide_numcols] = slide_cats[slide_numcols].apply(lambda row: pd.to_numeric(row, errors='coerce'),axis=0)
slide_cats

,Trick Name,Boardslide,Lipslide,50-50,Nose Slide,Tail Slide,Blunt Slide,Unnamed: 7
NaN,---,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,Back two in two out,0.0,0.0,0.0,0.0,0.0,0.0,NaN
NaN,Front lip,0.0,1.0,0.0,0.0,0.0,0.0,NaN
NaN,Cap two pull,0.0,0.0,0.0,0.0,0.0,0.0,NaN
NaN,Down flat down board slide,1.0,0.0,0.0,0.0,0.0,0.0,NaN
NaN,Back tail same way 270,0.0,0.0,0.0,0.0,1.0,0.0,NaN
NaN,Front blunt same way grab mute on the rail,0.0,0.0,0.0,0.0,0.0,1.0,NaN
NaN,Switch front blunt same way,0.0,0.0,0.0,0.0,0.0,1.0,NaN
NaN,Switch lip pretzel,0.0,1.0,0.0,0.0,0.0,0.0,NaN
NaN,Hard way cap two back to rag down rail,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [213]:
pd.concat([trick_df,slide_cats[slide_numcols]],axis=1)

,Trick Name,Grabs,Spins,Flips,Jumps,Slides,Buttering,Other,New Category,Unnamed: 9,Boardslide,Lipslide,50-50,Nose Slide,Tail Slide,Blunt Slide,Unnamed: 7
NaN,---,NaN,NaN,NaN,NaN,NaN,NaN,NaN,---,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,Back two in two out,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
NaN,Front lip,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN
NaN,Cap two pull,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
NaN,Down flat down board slide,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,NaN
NaN,Back tail same way 270,0.0,1.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,1.0,0.0,NaN
NaN,Front blunt same way grab mute on the rail,1.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,NaN
NaN,Switch front blunt same way,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,NaN
NaN,Switch lip pretzel,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN
NaN,Hard way cap two back to rag down rail,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [208]:
pd.concat([trick_df,slide_cats], on='Trick Name',how='left')

,Trick Name,Grabs,Spins,Flips,Jumps,Slides,Buttering,Other,New Category,Unnamed: 9,Boardslide,Lipslide,50-50,Nose Slide,Tail Slide,Blunt Slide,Unnamed: 7
0,---,NaN,NaN,NaN,NaN,NaN,NaN,NaN,---,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Back two in two out,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,Front lip,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN
3,Cap two pull,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,Down flat down board slide,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,NaN
5,Back tail same way 270,0.0,1.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,1.0,0.0,NaN
6,Front blunt same way grab mute on the rail,1.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,NaN
7,Switch front blunt same way,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,NaN
8,Switch front blunt same way,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,NaN
9,Switch lip pretzel,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN
